In [2]:
import pandas as pd

In [4]:
df = pd.concat([pd.read_csv("./datasets/d1.csv"), pd.read_csv("./datasets/d2.csv"), pd.read_csv("./datasets/d3.csv"), pd.read_csv("./datasets/d4.csv")])
print(df.shape)
df.head()

(1330246, 20)


,Unnamed: 0,body_type,latitude,longitude,daysonmarket,engine_displacement,frame_damaged,fuel_type,has_accidents,horsepower,listing_color,major_options,make_name,mileage,model_name,owner_count,price,transmission,wheel_system,year
0,0,Coupe,18.3467,-66.1098,1252,1700.0,False,Gasoline,False,237.0,RED,['Leather Seats'],Alfa Romeo,301.0,4C,2.0,97579.0,A,RWD,2015
1,1,Sedan,18.3467,-66.1098,1233,3000.0,False,Gasoline,False,320.0,SILVER,"['Sunroof/Moonroof', 'Alloy Wheels', 'Bluetooth']",BMW,6903.0,3 Series,2.0,58995.0,A,AWD,2016
2,2,Sedan,40.7333,-73.2587,33,1600.0,False,Gasoline,False,201.0,BLACK,"['Sport Package', 'Sunroof/Moonroof', 'Adaptiv...",Hyundai,30707.0,Elantra,1.0,13324.0,A,FWD,2017
3,3,Sedan,40.7333,-73.2587,55,1500.0,False,Gasoline,False,160.0,SILVER,"['Driver Confidence Package', 'Power Package',...",Chevrolet,42394.0,Malibu,1.0,14639.0,A,FWD,2018
4,4,Coupe,40.8847,-73.8317,36,3500.0,False,Gasoline,False,311.0,BLACK,"['Leather Seats', 'Sunroof/Moonroof', 'Navigat...",Lexus,62251.0,RC 350,1.0,32000.0,A,AWD,2018
